# Sustainability reports & NLP  
Thursday, March 17, 2022

In this hackathon learning session, we will walk through an example of parsing a pdf sustainability report and classifying it using one-shot learning in Python.  

---

## Concepts
#### Corporate Social Responsibility Reports (CSR)
A corporate social responsibility (CSR) report is an internal and external facing document companies use to communicate CSR efforts around environmental, ethical, philanthropic, and economic impacts on the environment and community.    

While they are not required in any sense, over 90% of S&P 500 index companies publish them anually [[source]](https://www.ga-institute.com/index.php?id=9128). As there's not a standard reporting process, the quantity and quality of information disclosed is up to the PR department at each company. The reports can be anywhere from 30 to 200+ pages.  

CSR reports are available to the public on a company's website or on [www.responsibilityreports.com/Company](www.responsibilityreports.com).


#### Natural Language Processing (NLP)
Natural language processing (NLP) is a field of linguistics and machine learning that deals with natural (i.e., human) languages. The goal is to "understand" the unstructured text data and produce something new. Examples of NLP tasks are language translation, text summarization, and sentiment analysis.  


#### Zero-Shot Learning (ZSL)
Human languages are really complex, so it is impossible to train classifiers on every single phrase. Zero-shot learning (ZSL) models allow classification of text into categories unseen by the model during training. These methods work by combining the observed/seen and the non-observed/unseen categories through auxiliary information, which encodes properties of objects.    

Other common uses for zero-shot learning models are images and videos. And the uses keep growing, such as activity recognition from sensors.

We will use NLP and ZSL to analyze a CSR report in order to classify each sentence as one of several categories relating to ESG.  

---

In [ ]:
!pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32892 sha256=2e4fd6bb8bc365a5f86dd286ec79f689a1b083b8f33c5b4598e6cfd745b2ee5f
  Stored in directory: /root/.cache/pip/wheels/75/66/8b/d1acbac7d49f3d98ade76c51ae5d72cec1866131a3b1ad9f82
Successfully built tika


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.0 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
     |████████████████████████████████| 7.6 MB 59.9 MB/s 


In [ ]:
# Imports
import re
import string
from collections import defaultdict
import pandas as pd
from tika import parser
import nltk
import torch
from transformers import pipeline  # Hugging Face

pd.set_option("display.max_colwidth", None)

## Parsing CSR PDFs
A non-trivial portion of classifying CSR reports is converting them to a computer-readable format. Companies publish their CSR reports as PDFs, which are notoriously hard to read. Our goal is to extract text as a list of sentences.  

We will be doing very simple parsing of a PDF report using the package tika to extract the text, regular expressions to filter and join the text, and NLTK to split the text into sentences.  

This is by no means the best way to do it, but it's relatively simple and gets the job done well enough for our purposes. Text cleaning is task-specific, so you need to consider what is sufficient for your problem.

In [ ]:
class parsePDF:
    def __init__(self, url):
        self.url = url

    def extract_contents(self):
        """ Extract a pdf's contents using tika. """
        pdf = parser.from_file(self.url)
        self.text = pdf["content"]
        return self.text


    def clean_text(self):
        """ Extract & clean sentences from raw text of pdf. """
        # Remove non ASCII characters
        printables = set(string.printable)
        self.text = "".join(filter(lambda x: x in printables, self.text))

        # Replace tabs with spaces
        self.text = re.sub(r"\t+", r" ", self.text)

        # Aggregate lines where the sentence wraps
        # Also, lines in CAPITALS is counted as a header
        fragments = []
        prev = ""
        for line in re.split(r"\n+", self.text):
            if line.isupper():
                prev = "."  # skip it
            elif line and (line.startswith(" ") or line[0].islower()
                  or not prev.endswith(".")):
                prev = f"{prev} {line}"  # make into one line
            else:
                fragments.append(prev)
                prev = line
        fragments.append(prev)

        # Clean the lines into sentences
        sentences = []
        for line in fragments:
            # Use regular expressions to clean text
            url_str = (r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\."
                       r"([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*")
            line = re.sub(url_str, r" ", line)  # URLs
            line = re.sub(r"^\s?\d+(.*)$", r"\1", line)  # headers
            line = re.sub(r"\d{5,}", r" ", line)  # figures
            line = re.sub(r"\.+", ".", line)  # multiple periods

            line = line.strip()  # leading & trailing spaces
            line = re.sub(r"\s+", " ", line)  # multiple spaces
            line = re.sub(r"\s?([,:;\.])", r"\1", line)  # punctuation spaces
            line = re.sub(r"\s?-\s?", "-", line)  # split-line words

            # Use nltk to split the line into sentences
            for sentence in nltk.sent_tokenize(line):
                s = str(sentence).strip().lower()  # lower case
                # Exclude tables of contents and short sentences
                if "table of contents" not in s and len(s) > 5:
                    sentences.append(s)
        return sentences

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##### Example: McDonald's
Here, we're pulling McDonalds' most recent CSR report from [responsibilityreports.com](https://www.responsibilityreports.com/Company/mcdonalds-corporation). We will extract and parse the text in order to move on to classifying it using zero shot learning.

In [ ]:
mcdonalds_url = "https://www.responsibilityreports.com/Click/2534"
pp = parsePDF(mcdonalds_url)
pp.extract_contents()
sentences = pp.clean_text()

print(f"The McDonalds CSR report has {len(sentences):,d} sentences")

2022-12-20 15:50:47,268 [MainThread  ] [INFO ]  Retrieving https://www.responsibilityreports.com/Click/2534 to /tmp/click-2534.
INFO:tika.tika:Retrieving https://www.responsibilityreports.com/Click/2534 to /tmp/click-2534.
2022-12-20 15:50:48,419 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-12-20 15:50:48,949 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-12-20 15:50:49,409 [MainThread  ] [WARNI]  Failed to see startup log message; retr

The McDonalds CSR report has 288 sentences


## Zero-Shot Learning
Zero-shot learning models are extremely helpful when you want to classify text on very specific labels and don't have labeled data. Labeled data can be difficult, expensive, and tedious to acquire, so zero-shot learning provides a quick way to get a classification without specialized data and additional model training.   

We are going to define industry-specific ESG categories and ask our model to classify each sentence in our CSR report. We will get a "score" that shows how confident the model is that that label applies. A score of 1.0 means that that sentence is definitely about that topic. Conversely, a score of 0.0 means that the sentence definitely doesn't relate to that topic.  

The downside to zero-shot learning is that it is extremely slow compared to models trained on specific labels. It basically has to compute "what it means to be that label" then it has to check if your sentence "is that label."

In [ ]:
class ZeroShotClassifier:

    def create_zsl_model(self, model_name):
        """ Create the zero-shot learning model. """
        self.model = pipeline("zero-shot-classification", model=model_name)


    def classify_text(self, text, categories):
        """
        Classify text(s) to the pre-defined categories using a
        zero-shot classification model and return the raw results.
        """
        # Classify text using the zero-shot transformers model
        hypothesis_template = "This text is about {}."
        result = self.model(text, categories, multi_label=True,
                            hypothesis_template=hypothesis_template)
        return result


    def text_labels(self, text, category_dict, cutoff=None):
        """
        Classify a text into the pre-defined categories. If cutoff
        is defined, return only those entries where the score > cutoff
        """
        # Run the model on our categories
        categories = list(category_dict.keys())
        result = (self.classify_text(text, categories))

        # Format as a pandas dataframe and add ESG label
        df = pd.DataFrame(result).explode(["labels", "scores"])
        df["ESG"] = df.labels.map(category_dict)

        # If a cutoff is provided, filter the dataframe
        if cutoff:
            df = df[df.scores.gt(cutoff)].copy()
        return df.reset_index(drop=True)

##### Pre-Define Labels
The labels chosen below are based on categories and topics used by ESG scoring companies.  
We define the plain-english version, which is what will be searched by the zero-shot learning model, as well as the general "ESG" label.  

Because of how zero-shot learning models work, inference time will increase linearly with the number of labels you define. Therefore, it is necessary to consider which labels you really want and how much time is acceptable for text classification.

In [ ]:
# Define categories we want to classify
esg_categories = {
  "emissions": "E",
  "natural resources": "E",
  "pollution": "E",
  "diversity and inclusion": "S",
  "philanthropy": "S",
  "health and safety": "S",
  "training and education": "S",
  "transparancy": "G",
  "corporate compliance": "G",
  "board accountability": "G"}

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 12.8 MB/s 


##### Getting Text Classification
Now, all we have to do is define the model and make predictions. The architecture of the model can be chosen from any text-classification model on [Hugging Face](https://huggingface.co/models).  

Here, we choose to use the extra large version of the DeBERTa model, as maintained by Microsoft. A larger model (generally) gives better performance but is much slower.

In [ ]:
# Define and Create the zero-shot learning model
model_name = "microsoft/deberta-v2-xlarge-mnli"
    # a smaller version: "microsoft/deberta-base-mnli"
ZSC = ZeroShotClassifier()
ZSC.create_zsl_model(model_name)
    # Note: the warning is expected, so ignore it

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Classify all the sentences in the report
    # Note: this takes a while
classified = ZSC.text_labels(sentences, esg_categories)
classified.sample(n=20)  # display 20 random records

,sequence,labels,scores,ESG
503,.8%7 of primary fiber-based guest packaging continues to be sourced from recycled orcertified sources.,emissions,0.187001,E
190,"as communities around the world experience the impacts of climate change, we believe we need to be part of the solution.",emissions,0.849404,E
2049,"in 2021, we raised hourly wages by an average of 10% for more than 36,500 employees at company-owned u.s. restaurants.",pollution,0.000741,E
665,"in 2021, we made notable progress in ensuring standards were upheld, including: 2,000+ farmers trained on mcdonalds good agricultural practices standards, which cover food safety as well as topics such as soil health, water use and land management.",diversity and inclusion,0.020237,S
598,"our 2021 progress at a glance purpose & impact progress summary introduction food quality & sourcing our planet jobs, inclusion & empowerment community connection 4 jobs, inclusion & empowerment diversity, equity & inclusion page 12 we published the results of our 2021 paygap analysis and have taken steps to close the identified gaps for women at the staff and company-owned restaurant levels in company-owned and-operated markets.",health and safety,0.000413,S
127,"these actions define what it means towork at mcdonalds, to be part of themcfamily and to get better, together.",emissions,0.000832,E
448,"our 2021 progress at a glance purpose & impact progress summary introduction food quality & sourcing our planet jobs, inclusion & empowerment community connection 3 global happy meal goals 2020 progress global happy meal goals 2020 progress our planet climate action page 8 we have achieved a 2.9%7 reduction in the absolute greenhouse gas (ghg) emissions of our restaurants and offices compared to 2015 figures.",health and safety,0.000776,S
1901,"talent & benefits jobs, inclusion & empowerment delivering best-in-class employeeexperiences to deliver an exceptional employee experience, we must put our five core values into action, measuring progress through two dedicated indices.",transparancy,0.007846,G
10,"guided by our core values, weve experienced first-hand how our focused actions both big and small can translate into meaningful experiences for our customers, bringing our purpose to feed and foster community to life each day.",diversity and inclusion,0.880213,S
1400,were aiming for a 90% reduction in virgin fossil fuel-based plastic used to make happy meal toys by the end of 2025.,natural resources,0.989216,E


In [ ]:
# Look at an example of "E" classified sentences:
E_sentences = classified[classified.scores.gt(0.8) & classified.ESG.eq("E")].copy()
E_sentences.head()

,sequence,labels,scores,ESG
170,helping protect our planet earning the trust of our people and customers by doing what we say weregoing to do has always been key to building a strong brand and a lasting legacy.,natural resources,0.960891,E
190,"as communities around the world experience the impacts of climate change, we believe we need to be part of the solution.",emissions,0.849404,E
200,"thats why, in 2021, we set an ambition to achieve net zero emissions by 2050.",emissions,0.993149,E
201,"thats why, in 2021, we set an ambition to achieve net zero emissions by 2050.",pollution,0.951108,E
210,"were prioritizing action on the largest elements of our carbon footprint from restaurant energy use to packaging and waste, and the sourcing of key ingredients for our menu.",emissions,0.990858,E
